In [ ]:
import ee


In [ ]:
ee.Initialize()

In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import geemap

In [ ]:
Map = geemap.Map(center=[19.2706, 84.8037], zoom=10, max_pixels=1e13)

In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium

In [ ]:
roi = ee.Geometry.Polygon(  [[84.9392,19.1501], [84.9392,19.3912],[84.5767,19.3912],[84.5767,19.1501]] )

# Define the Landsat 8 SR image collection
img_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter for the year 2020 and the region of interest
filtered = img_collection \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(roi)) \

def cloud_mask_l8(image):
    # Select the pixel QA band
    qa = image.select('pixel_qa')
    # Make a mask of clear and water pixels
    clear = qa.bitwiseAnd(int('0000000000000010', 2)).neq(0)
    water = qa.bitwiseAnd(int('0000000000000100', 2)).neq(0)
    # Mask out cloudy or shadowed pixels
    masked = image.updateMask(clear.Or(water))
    return masked

cloud_masked = filtered.map(cloud_mask_l8)

def addSpectralIndices(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI')
  mndwi = image.normalizedDifference(['B3', 'B6']).rename('MNDWI')
  bsi = image.expression(
      '((SWIR1 + NIR) - (SWIR2 + BLUE)) / ((SWIR1 + NIR) + (SWIR2 + BLUE))',
      {
        'SWIR1': image.select('B6'),
        'NIR': image.select('B5'),
        'SWIR2': image.select('B7'),
        'BLUE': image.select('B2')
      }).rename('BSI')
  return image.addBands([ndvi, ndbi, mndwi, bsi])


composite = cloud_masked.median()
composite_2020 = addSpectralIndices(composite)


#palette = [
 # 'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  #'74A901', '66A000', '529400', '3E8601', '207401', '056201',
  #'004C00', '023B01', '012E01', '011D01', '011301']

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Display the composite image
Map.centerObject(roi, 10)
Map.addLayer(composite_2020.clip(roi), vis_params, 'Landsat 8 SR 2020')
Map

Map(center=[19.27070975400328, 84.75794999999981], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:

features = ee.FeatureCollection('projects/ee-sharmavinayak4u/assets/assetIDLULC_roi')

label = "lulc"
bands = ['B1','B2','B3','B4','B5','B6','B7']
input = composite.clip(roi).select(bands)
features=features.randomColumn()


In [ ]:
#train_test_split
trainSet = features.filter(ee.Filter.lessThan('random',0.7))
testSet = features.filter(ee.Filter.greaterThanOrEquals('random',0.7))
testSet.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [84.76209335539923, 19.324092677918543]},
 'id': '00000000000000000000',
 'properties': {'lulc': 1, 'random': 0.9995321853167387}}

In [ ]:
#overlay the points i=on the imagery to get training
training= input.sampleRegions(**{
    'collection': trainSet,
    'properties': [label],
    'scale': 30
})



#trainingData = trainImage.randomColumn()


In [ ]:
#define paletter
landcoverPalette = [
    '#2850d6',#water                                 #205ad6
    '#ff0707', #forest                               #75f231
    '#38e529', #urban                                #ff1717
    '#ffef4d', #cropland                             #e1a744
    '#d1c0c6', #barren                               #b1c2ae
]

### Random Forest

In [ ]:
#classification model
#classifier = ee.Classifier.smileCart().train(trainSet,label,bands)
#classifiy
#classified = input.classify(classifier)
#print(classified.getInfo())
rf_classifier = ee.Classifier.smileRandomForest(5).train(training,label,bands)
rf_classified = input.select(bands).classify(rf_classifier)

In [ ]:

train_accuracy = rf_classifier.confusionMatrix()
train_accuracy.getInfo()


[[0, 0, 0, 0, 0, 0],
 [0, 56, 0, 0, 0, 0],
 [0, 0, 59, 1, 3, 0],
 [0, 0, 0, 60, 0, 0],
 [0, 0, 2, 0, 60, 1],
 [0, 0, 2, 1, 1, 54]]

In [ ]:
train_accuracy.accuracy().getInfo()

0.9633333333333334

In [ ]:
train_accuracy.kappa().getInfo()

0.9541367281419816

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9365079365079365],
 [1],
 [0.9523809523809523],
 [0.9310344827586207]]

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

[[0, 1, 0.9365079365079365, 0.967741935483871, 0.9375, 0.9818181818181818]]

**Validation Accuracy**

In [ ]:
test = rf_classified.sampleRegions(**{
    'collection': testSet,
    'properties': [label],
    'scale': 30
})
test_confusion_matrix = ee.ConfusionMatrix(test.errorMatrix('lulc', 'classification'))

# Print the confusion matrix and accuracy
print('Confusion Matrix', test_confusion_matrix.getInfo())
print('Test Accuracy', test_confusion_matrix.accuracy().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0], [0, 23, 0, 0, 1, 0], [0, 0, 20, 2, 2, 3], [0, 2, 0, 24, 0, 0], [0, 0, 1, 4, 28, 1], [0, 0, 4, 0, 2, 21]]
Test Accuracy 0.8405797101449275


### SVM

In [ ]:
svm_classifier = ee.Classifier.libsvm( decisionProcedure='voting',
    kernelType='LINEAR',
    shrinking=True,
    cost=4,
     ).train(training,label,bands)
svm_classified = input.select(bands).classify(svm_classifier)

In [ ]:

train_accuracy = svm_classifier.confusionMatrix()
train_accuracy.getInfo()
#train_confusion_matrix = svm_classifier.confusionMatrix()

# Print the confusion matrix and accuracy
#print('Confusion Matrix', train_confusion_matrix.getInfo())
#print('Training Accuracy', train_confusion_matrix.accuracy().getInfo())

[[0, 0, 0, 0, 0, 0],
 [0, 56, 0, 0, 0, 0],
 [0, 0, 59, 0, 4, 0],
 [0, 1, 0, 56, 3, 0],
 [0, 0, 7, 3, 52, 1],
 [0, 0, 1, 0, 2, 55]]

In [ ]:
train_accuracy.accuracy().getInfo()

0.9266666666666666

In [ ]:
train_accuracy.kappa().getInfo()

0.9082772805603425

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9365079365079365],
 [0.9333333333333333],
 [0.8253968253968254],
 [0.9482758620689655]]

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9824561403508771,
  0.8805970149253731,
  0.9491525423728814,
  0.8524590163934426,
  0.9821428571428571]]

**Validation Accuracy**

In [ ]:
test = svm_classified.sampleRegions(**{
    'collection': testSet,
    'properties': [label],
    'scale': 30
})
test_confusion_matrix = ee.ConfusionMatrix(test.errorMatrix('lulc', 'classification'))

# Print the confusion matrix and accuracy
print('Confusion Matrix', test_confusion_matrix.getInfo())
print('Test Accuracy', test_confusion_matrix.accuracy().getInfo())


Confusion Matrix [[0, 0, 0, 0, 0, 0], [0, 23, 0, 0, 1, 0], [0, 0, 26, 0, 1, 0], [0, 1, 0, 23, 2, 0], [0, 0, 1, 4, 29, 0], [0, 0, 1, 0, 0, 26]]
Test Accuracy 0.9202898550724637


### CART

In [ ]:
cart_classifier = ee.Classifier.smileCart(maxNodes=50,
    minLeafPopulation=9).train(training,label,bands)
cart_classified = input.select(bands).classify(cart_classifier)

In [ ]:

train_accuracy = cart_classifier.confusionMatrix()
train_accuracy.getInfo()


[[0, 0, 0, 0, 0, 0],
 [0, 55, 1, 0, 0, 0],
 [0, 0, 51, 0, 10, 2],
 [0, 2, 0, 52, 6, 0],
 [0, 0, 5, 0, 58, 0],
 [0, 0, 5, 0, 6, 47]]

In [ ]:
train_accuracy.accuracy().getInfo()

0.8766666666666667

In [ ]:
train_accuracy.kappa().getInfo()

0.8456189151599444

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9821428571428571],
 [0.8095238095238095],
 [0.8666666666666667],
 [0.9206349206349206],
 [0.8103448275862069]]

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

[[0, 0.9649122807017544, 0.8225806451612904, 1, 0.725, 0.9591836734693877]]

**Validation Accuracy**

In [ ]:
test = cart_classified.sampleRegions(**{
    'collection': testSet,
    'properties': [label],
    'scale': 30
})
test_confusion_matrix = ee.ConfusionMatrix(test.errorMatrix('lulc', 'classification'))

# Print the confusion matrix and accuracy
print('Confusion Matrix', test_confusion_matrix.getInfo())
print('Test Accuracy', test_confusion_matrix.accuracy().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0], [0, 23, 1, 0, 0, 0], [0, 0, 20, 1, 4, 2], [0, 1, 0, 21, 4, 0], [0, 0, 2, 1, 31, 0], [0, 0, 2, 0, 1, 24]]
Test Accuracy 0.8623188405797102


### Map Visualization

In [ ]:
#define a method for displaying ee image tiles to folium map
def add_ee_layer(self,ee_image_object,vis_p,name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_p)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
    ).add_to(self)

In [ ]:
#add ee drawing method to folium
folium.Map.add_ee_layer = add_ee_layer
# create a folium map object
my_map = folium.Map(location=[19.58, 84.68],zoom_start = 10)

cl_vis_params= {
    'palette': landcoverPalette,
    'min': 1,
    'max': 5,
        
}
#add layer to the map
combined_lulc=composite.clip(roi).visualize(**vis_params)
combined_classification=svm_classified.visualize(**cl_vis_params)
my_map.add_ee_layer(combined_lulc,{},"landsat 2020")
my_map.add_ee_layer(combined_classification,{},"classification")

#add control panel
my_map.add_child(folium.LayerControl())
display(my_map)

In [ ]:
  # Define your region of interest

export_params = {
    'image': combined_classification,
    'description': 'classification_SVM_',
    'bucket': 'lulcbucket',  # Replace with your GCS bucket name
    'fileFormat': 'auto',
    'maxZoom': 13,
    'region': roi,  # Use the extracted geometry as the export region
    'writePublicTiles': True
}  # Define your export parameters as a dictionary object

task = ee.batch.Export.map.toCloudStorage(**export_params)  # Call the Export.map.toCloudStorage() function with the export parameters
task.start()  # Start the export task

In [ ]:

export_params = {
  'image': combined_classification,
  'description': 'classification_SVM_image_asset',
  'assetId': 'projects/ee-sharmavinayak4u/assets/ASSET_ID_LULC_Main',
  'pyramidingPolicy': {
    '.default': 'mode'
  },
  'region': geometry,
  'scale': 30,
  'maxPixels': 1e13
}

# Export the image to an Asset
task = ee.batch.Export.image.toAsset(**export_params)
task.start()

In [ ]:
# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

composite = withNdvi.median()

palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']

ndviVis = {'min':0, 'max':0.5, 'palette': palette }
Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')
Map